In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .remote("sc://192.168.1.7:15002") \
  .appName("UDFTransformation") \
  .config("spark.sql.ansi.enabled", "false") \
  .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
  .config("spark.sql.repl.eagerEval.enabled", "true") \
  .config("spark.sql.repl.eagerEval.truncate", "100") \
  .getOrCreate()

spark

In [18]:
from pyspark.sql.utils import IllegalArgumentException

lines = (spark
  .readStream
  .format("socket")
  .option("host", "192.168.1.7")
  .option("port", 9999)
  .load()
)

# lines = (spark.readStream
#   .format("rate")
#   .option("rowsPerSecond", 3)
#   .load()
# )
try:
  query = (lines.writeStream
    .outputMode("update")
    .format("memory")
    .queryName("mem_results")
    .start()
  )
except IllegalArgumentException:
  pass # already started
import time

if True:
  for _ in range(100):
    print(f"{str(query.status):<300}", end="\r")
    time.sleep(0.1) 

In [19]:
spark.sql("show tables")

namespace,tableName,isTemporary
,mem_results,true


In [21]:
spark.sql("select * from mem_results order by value desc")

value
test2
now2
now


In [ ]:
if True:
  query.stop()